In [1]:
import numpy as np
from multi_funcs import *
from multi_funcs_2 import *
import pandas as pd
from indiv_cycles import *

In [2]:
results = ic_simulate_par(1000, 6, 100)

In [3]:
make_df_ic(results)

,cycle_length,id
3,14,0
4,13,0
6,8,0
10,7,0
12,8,0
...,...,...
82191,20,99
82205,12,99
82231,12,99
82246,19,99


In [62]:
preferences = mdf_np(1000, 3)
preferences2 =preferences.copy()
matches, _ = run_gale_shapley(preferences, 3)

In [63]:
matches

,0,1,2,student_id,applications,k,matched,rank1,rank2,rank3,underdemanded
0,592,951,69,1,0,1,False,0.934009,0.189201,0.511521,False
1,362,405,405,2,1,1,False,0.939373,0.572059,0.572059,False
2,800,582,311,3,0,1,False,0.524542,0.600676,0.947191,False
3,759,759,759,5,2,1,False,0.930900,0.930900,0.930900,True
4,91,91,91,7,2,1,False,0.301047,0.301047,0.301047,True
...,...,...,...,...,...,...,...,...,...,...,...
836,498,44,44,992,1,1,False,0.693109,0.888508,0.888508,False
837,872,872,872,993,2,1,False,0.351578,0.351578,0.351578,True
838,913,24,664,994,0,1,False,0.042905,0.782927,0.323105,True
839,271,811,284,997,0,1,False,0.871283,0.006219,0.390740,True


In [64]:
def to_edgelist(preferences, matches, k):
    '''
    Function that takes as an input the results of a simulation and returns the number
    of cycles in the GS result.
    _______
    Inputs:
    preference: the initial preference dataframe
    matches: the GS matches dataframe
    '''    
    ## Step 1: drop all unmatched students from the preference dataframe.
    preferences = preferences[preferences['student_id'].isin(matches['student_id'])]
    preferences.reset_index(inplace = True, drop = True) 
    preferences['rejections'] = matches.applications

    relevant = preferences.copy()
    
    ## Step 2: Remove students who got preferred choice
    #relevant = preferences[preferences['rejections'] != 0] ## drop all who were never rejected. They will necessarily not point to anyone else. 
    
    ## Step 3: For others, keep only preferences above match. 
    for i in range(1,k):
        relevant.iloc[:, i] = np.where(relevant['rejections']<i+1, -100, relevant.iloc[:, i])
    relevant.set_index('student_id', inplace = True)
    pointing = pd.DataFrame(relevant.iloc[:, :k].stack(level = 0)).reset_index()
    pointing = pointing[pointing[0] != -100]
    
    ## Step 4: Prepare for Tarjan's Algorithm
    to_merge = matches.loc[:,[0, 'student_id']]
    pointing = pointing.merge(to_merge, on = 0, how = 'left')
    pointing = pointing[pointing['student_id_y'].isin(pointing['student_id_x'])]
    normalizer = pd.DataFrame(pd.concat([pointing['student_id_x'], pointing['student_id_y']], axis = 0).unique())
    normalizer['new_id'] = normalizer.index
    pointing = pointing.merge(normalizer, left_on = 'student_id_x', right_on = 0, how = 'left')
    pointing = pointing.merge(normalizer, left_on = 'student_id_y', right_on = 0, how = 'left')
    pairs = pointing[['new_id_x', 'new_id_y']]
    pairs = pairs[pairs['new_id_x'] != pairs['new_id_y']]
    return pairs

In [67]:
edgelist = to_edgelist(preferences, matches, 3)

In [68]:
edgelist

,new_id_x,new_id_y
1,1,786
3,3,773
5,4,324
8,6,455
23,21,532
...,...,...
942,816,221
956,829,488
961,834,115
962,834,450


In [69]:
G= nx.from_pandas_edgelist(edgelist, source = 'new_id_x', target = 'new_id_y', create_using=nx.DiGraph())

In [72]:
list(all_pairs_shortest_cycle_length(G).values())

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [55]:
import networkx as nx

def all_pairs_shortest_cycle_length(G):
    # Compute all pairs shortest path lengths
    shortest_path_lengths = nx.all_pairs_shortest_path_length(G)
    
    shortest_path_lengths = dict(shortest_path_lengths)

    # Dictionary to store shortest cycle lengths
    shortest_cycle_lengths = {}

    # Iterate over each node in the graph
    for i in G.nodes():
        shortest_cycle = float('inf')
        
        # Check one-hop edges from node i
        for j in G.successors(i):
            # Check if there is a shortest path from j back to i
            if i in shortest_path_lengths[j]:
                cycle_length = shortest_path_lengths[j][i] + 1
                shortest_cycle = min(shortest_cycle, cycle_length)
        
        # If shortest_cycle is still infinity, no cycle found
        if shortest_cycle == float('inf'):
            shortest_cycle_lengths[i] = -1
        else:
            shortest_cycle_lengths[i] = shortest_cycle
            
    return shortest_cycle_lengths
